In [36]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config('spark.shuffle.useOldFetchProtocol', 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [37]:
orders_schema = "order_id long , order_date string, cust_id long,order_status string"

In [38]:
orders_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("/public/trendytech/orders/orders_1gb.csv")

In [4]:
spark.conf.get("spark.sql.files.maxPartitionBytes")

'134217728b'

In [5]:
134217728 / (1024 * 1024)

128.0

In [6]:
spark.sparkContext.defaultParallelism

2

In [8]:
orders_df.rdd.getNumPartitions()

9

In [39]:
new_orders_df = orders_df.repartition(1)

In [11]:
new_orders_df.rdd.getNumPartitions()

1

In [12]:
new_orders_df.write \
.format("csv") \
.mode("overwrite") \
.option("codec", "org.apache.hadoop.io.compress.GzipCodec") \
.save("Week9 - Spark Internals and Writer API/orders_gz")

In [13]:
orders_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("Week9 - Spark Internals and Writer API/orders_gz")

In [14]:
orders_df.show()

+--------+--------------------+-------+---------------+
|order_id|          order_date|cust_id|   order_status|
+--------+--------------------+-------+---------------+
|   12649|2013-10-11 00:00:...|   2789|PENDING_PAYMENT|
|   12650|2013-10-11 00:00:...|   4566|         CLOSED|
|   12651|2013-10-11 00:00:...|    960|       COMPLETE|
|   12652|2013-10-11 00:00:...|   8251|PENDING_PAYMENT|
|   12653|2013-10-11 00:00:...|   5836|        PENDING|
|   12654|2013-10-11 00:00:...|    499|         CLOSED|
|   12655|2013-10-11 00:00:...|   2644|       COMPLETE|
|   12656|2013-10-11 00:00:...|    933|        PENDING|
|   12657|2013-10-11 00:00:...|   1835|     PROCESSING|
|   12658|2013-10-11 00:00:...|   1868|       COMPLETE|
|   12659|2013-10-11 00:00:...|    118|     PROCESSING|
|   12660|2013-10-11 00:00:...|  11615|PENDING_PAYMENT|
|   12661|2013-10-11 00:00:...|   9670|        PENDING|
|   12662|2013-10-11 00:00:...|   6465|        PENDING|
|   12663|2013-10-11 00:00:...|  11233|        P

In [16]:
orders_df.rdd.getNumPartitions()

1

In [17]:
new_orders_df.write \
.format("csv") \
.mode("overwrite") \
.option("codec", "snappy") \
.save("Week9 - Spark Internals and Writer API/orders_snappy")

In [18]:
orders_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("Week9 - Spark Internals and Writer API/orders_snappy")

In [19]:
orders_df.rdd.getNumPartitions()

1

In [20]:
new_orders_df.write \
.mode("overwrite") \
.save("Week9 - Spark Internals and Writer API/orders_parquet")

In [21]:
orders_df = spark.read \
.load("Week9 - Spark Internals and Writer API/orders_parquet")

In [24]:
orders_df.rdd.getNumPartitions()

7

In [25]:
new_orders_df.rdd.getNumPartitions()

1

In [40]:
final_orders_df = new_orders_df.repartition(20)

In [28]:
final_orders_df.rdd.getNumPartitions()

20

In [29]:
final_orders_df.write \
.format("csv") \
.mode("overwrite") \
.save("Week9 - Spark Internals and Writer API/orders_final")

In [30]:
orders_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("Week9 - Spark Internals and Writer API/orders_final")

In [31]:
orders_df.rdd.getNumPartitions()

10